In [10]:
import pandas as pd
import numpy as np
import time
import math
from datetime import datetime

In [11]:
N = 3 # number of lanes
FT_MI = 5280 # mile to feet conversion
TAO = 1.75 #
SPEED_CONVER = 5280/3600 # ft/s to mph conversion
L = 14.75 # typical length of a car
l = 5 # loop length
DEPARTURE_CONVER = 120 # hourly departure of v vehicles; raw data shows vehicles per 30 seconds

In [12]:
# tweaking settings to see all columns on screen
pd.set_option('display.max_columns', None)

# Data Cleaning

In [24]:
df = pd.read_csv('78 EB.csv')

# Record the Date when the data was recorded
# date = df.loc[0, 'Date']
# date = datetime.strptime(date, '%Y/%m/%d')

# df.drop(columns=['Route', 'Direction', 'pcw1occ', 'Mainline Lanes', 'Date'], inplace=True)
df = df[['Time', 'Id', 'Route', 'Direction', 'Name', 'MlDetPostmile', 'pcw1occ', 'pcw1speed', 'pcw1vol', 'r_1_offcnt', 'r_2_offcnt', 'Psg 1 Cnt', 'Psg 2 Cnt', 'Psg 3 Cnt']]
# rename columns
df.rename(columns={"pcw1speed": "V_ups",
                   "pcw1vol": "Q_ups",
                   "MlDetPostmile": "postmile"}, inplace = True)

# drop rows whose Name value does not include the word 'to'
df = df[df.Name.str.contains('to')] # those that dont have 'to' is radar data; no ramp data
df.loc[:, 'Name'] = df.loc[:, 'Name'].apply(lambda x: x.split('to')[0][:-1])

# convert datatype of 'time' to datetime 
df['Time'] = df['Time'].apply(lambda s: datetime.strptime(s, '%H:%M:%S').time())

In [25]:
df

,Time,Id,Route,Direction,Name,postmile,pcw1occ,V_ups,Q_ups,r_1_offcnt,r_2_offcnt,Psg 1 Cnt,Psg 2 Cnt,Psg 3 Cnt
1,05:30:00,201,78,EB,Jefferson St,0.8580,1.4,73,213,0.0,0.0,0.0,0.0,0.0
2,05:30:00,200,78,EB,El Camino Real,1.5940,2.5,63,335,1.0,NaN,3.0,3.0,NaN
4,05:30:00,199,78,EB,Plaza Dr,3.5860,2.1,59,274,0.0,0.0,0.0,0.0,NaN
5,05:30:00,24,78,EB,Emerald Dr,4.4740,3.0,52,335,0.0,0.0,0.0,0.0,NaN
6,05:30:00,205,78,EB,Vista Village Dr,6.3200,3.0,79,517,0.0,0.0,0.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16812,19:30:00,180,78,EB,San Marcos Blvd,12.2740,0.0,0,0,0.0,0.0,0.0,0.0,0.0
16813,19:30:00,234,78,EB,Twin Oaks Vlly Rd,13.0220,6.2,64,853,1.0,3.0,0.0,2.0,2.0
16814,19:30:00,236,78,EB,Barham/Woodland,14.8600,5.9,67,853,NaN,NaN,0.0,1.0,NaN
16815,19:30:00,13006,78,EB,Nordahl Rd,15.5960,4.5,66,639,1.0,2.0,7.0,9.0,NaN


# ⚠️ WE ARE CONVERTING ALL NaN VALUES WITH 0.
# BOTTOM CELL REPRESENTS WHAT PERCENTAGE OF THE ROWS CONTAIN AT LEAST ONE NaN VALUE IN ['r_1_offcnt', 'r_2_offcnt', 'Psg 1 Cnt', 'Psg 2 Cnt', 'Psg 3 Cnt']
# SHOULD WE KEEP IT THIS WAY?

In [14]:
total_rows = df.shape[0]
na_rows = df[['r_1_offcnt', 'r_2_offcnt', 'Psg 1 Cnt', 'Psg 2 Cnt', 'Psg 3 Cnt']].dropna().shape[0]

round((total_rows - na_rows) / total_rows * 100, 2)

66.67

In [15]:
# convert Nan values to 0
df['r_1_offcnt'].fillna(0, inplace=True)
df['r_2_offcnt'].fillna(0, inplace=True)
df['Psg 1 Cnt'].fillna(0, inplace=True)
df['Psg 2 Cnt'].fillna(0, inplace=True)
df['Psg 3 Cnt'].fillna(0, inplace=True)

# ⚠️ WE ARE CONVERTING ALL NaN VALUES WITH 0.
# BOTTOM CELL REPRESENTS WHAT PERCENTAGE OF THE ROWS CONTAIN AT LEAST ONE NaN VALUE IN ['r_1_offcnt', 'r_2_offcnt', 'Psg 1 Cnt', 'Psg 2 Cnt', 'Psg 3 Cnt']
# SHOULD WE KEEP IT THIS WAY?

In [16]:
# Derive q_exit
df['q_exit'] = (df['r_1_offcnt'] + df['r_2_offcnt']) * DEPARTURE_CONVER
df.drop(['r_1_offcnt', 'r_2_offcnt'], axis=1, inplace = True)

# Derive q_ramp
df['q_ramp'] = (df['Psg 1 Cnt'] + df['Psg 2 Cnt'] + df['Psg 3 Cnt']) * DEPARTURE_CONVER
df.drop(['Psg 1 Cnt', 'Psg 2 Cnt', 'Psg 3 Cnt'], axis=1, inplace = True)

# Derive V_merge
V_ups = df['V_ups']
Q_ups = df['Q_ups']
q_ramp = df['q_ramp']

df['V_merge'] = (FT_MI*N*V_ups)/(TAO*SPEED_CONVER*((N*Q_ups)+q_ramp))-(L/(TAO*SPEED_CONVER))
df['V_merge'] = df[['V_ups','V_merge']].min(axis=1)

# ⚠️ convert Nan values to 0
df['q_exit'] = df['q_exit'].fillna(0)
df['q_ramp'] = df['q_ramp'].fillna(0)
df['V_ups'] = df['V_ups'].fillna(0)
df['Q_ups'] = df['Q_ups'].fillna(0)

# ⚠️ WE ARE CONVERTING ALL V_UPS VALUES EQUAL TO 0 WITH THE PRECEEDING V_UPS VALUE.
# BOTTOM CELL REPRESENTS WHAT PERCENTAGE OF THE ROWS OF ID=203 CONTAINS V_UPS == 0
# SHOULD WE KEEP IT THIS WAY?

In [17]:
row_per_Id = df[df['Id']==203].shape[0]
row_with_0_V_ups = df[(df['V_ups']==0) & (df['Id']==203)].shape[0]
round(row_with_0_V_ups / row_per_Id * 100, 2)

47.32

In [18]:
# change all V_ups value of 0 to the previous data
df.sort_values(['Time', 'postmile'], inplace=True)
df['V_ups'] = df['V_ups'].replace(to_replace=0, method='ffill')

df

,Time,Id,Route,Direction,Name,postmile,pcw1occ,V_ups,Q_ups,q_exit,q_ramp,V_merge
1,05:30:00,201,78,EB,Jefferson St,0.8580,1.4,73,213,0.0,0.0,73.0
2,05:30:00,200,78,EB,El Camino Real,1.5940,2.5,63,335,120.0,720.0,63.0
4,05:30:00,199,78,EB,Plaza Dr,3.5860,2.1,59,274,0.0,0.0,59.0
5,05:30:00,24,78,EB,Emerald Dr,4.4740,3.0,52,335,0.0,0.0,52.0
6,05:30:00,205,78,EB,Vista Village Dr,6.3200,3.0,79,517,0.0,600.0,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16812,19:30:00,180,78,EB,San Marcos Blvd,12.2740,0.0,84,0,0.0,0.0,0.0
16813,19:30:00,234,78,EB,Twin Oaks Vlly Rd,13.0220,6.2,64,853,480.0,480.0,64.0
16814,19:30:00,236,78,EB,Barham/Woodland,14.8600,5.9,67,853,0.0,120.0,67.0
16815,19:30:00,13006,78,EB,Nordahl Rd,15.5960,4.5,66,639,360.0,1920.0,66.0


In [19]:
# Rearrange table
df = df[['Time', 'Id', 'Name', 'postmile', 'V_ups', 'Q_ups', 'q_ramp', 'q_exit', 'V_merge']]
df.sort_values(by=['Time', 'postmile'], inplace=True)
df.reset_index(drop=True, inplace=True)

# Rename table
raw = df
df = None

# V_ups: upstream speed in mph
# Q_ups: upstream volume in lane/hr
# q_ramp: discharge rate; rate at which cars enter the highway per hour at a certain ramp
# q_exit: rate at which cars exit a certain ramp
# postmile: number of miles a ramp is into the highway

/Users/taekunkim/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
# Shift up q_exit values and calculate V_ds and Q_ds
# To do a calculation for ramp1 at time T, we need q_exit value of ramp2 at T+(1min)
# V_ds and Q_ds of ramp 1 at time T are V_ups and Q_ups of ramp 2 at time T+(1min)

first_time = raw['Time'].unique()[0] # first minute mark at which data was recorded
first_ramp = raw['Id'].unique()[0] # first ramp of the data
last_time = raw['Time'].unique()[-1]
last_ramp = raw['Id'].unique()[-1]

df = raw.copy()
df['q_exit_prev'] = 0
df = df[(df['Time'] != first_time) & (df['Id'] != first_ramp)].reset_index(drop=True)

temp_df = raw.copy()
temp_df = temp_df[(temp_df['Id'] != last_ramp) & (temp_df['Time'] != last_time)].reset_index(drop=True)
temp_df = temp_df.rename(columns={'V_ups': 'V_ds', 
                                  'Q_ups': 'Q_ds',
                                  'q_exit': 'q_exit_prev'})

df['V_ds'] = temp_df['V_ds']
df['Q_ds'] = temp_df['Q_ds']
df['q_exit_prev'] = temp_df['q_exit_prev']

# Rename table
raw = df
df = None



In [21]:
raw = pd.concat([raw, raw])

In [22]:
raw

,Time,Id,Name,postmile,V_ups,Q_ups,q_ramp,q_exit,V_merge,q_exit_prev,V_ds,Q_ds
0,05:31:00,200,El Camino Real,1.5940,65,243,480.0,0.0,65.0,0.0,73,213
1,05:31:00,199,Plaza Dr,3.5860,60,335,120.0,0.0,60.0,120.0,63,335
2,05:31:00,24,Emerald Dr,4.4740,70,396,360.0,120.0,70.0,0.0,59,274
3,05:31:00,205,Vista Village Dr,6.3200,76,396,600.0,120.0,76.0,0.0,52,335
4,05:31:00,198,Escondido Ave,6.8860,79,670,0.0,120.0,79.0,0.0,79,517
...,...,...,...,...,...,...,...,...,...,...,...,...
11755,19:30:00,180,San Marcos Blvd,12.2740,84,0,0.0,0.0,0.0,360.0,82,975
11756,19:30:00,234,Twin Oaks Vlly Rd,13.0220,64,853,480.0,480.0,64.0,0.0,66,761
11757,19:30:00,236,Barham/Woodland,14.8600,67,853,120.0,0.0,67.0,480.0,68,914
11758,19:30:00,13006,Nordahl Rd,15.5960,66,639,1920.0,360.0,66.0,0.0,67,670


# Tensorflow

In [15]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import matplotlib.pyplot as plt
from tqdm import tqdm
from keras.preprocessing import image
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

### Build Dataset

In [16]:
# split training set
X = raw[['Q_ups', 'V_ups', 'q_exit_prev', 'q_ramp']]
# y = raw[['Q_ds', 'V_ds']]
# y = raw[['V_ds']]
y = raw[['Q_ds']]



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [17]:
# Fit model
def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(4, activation='relu'),
        tf.keras.layers.Dense(4, activation='elu'),
        tf.keras.layers.Dense(4, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    
    tf.keras.optimizers.Adamax(
        learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07,
        name='Adamax'
    )
    
    model.compile(optimizer='adamax',
                  loss='mean_absolute_error',
                  metrics=[tf.keras.metrics.RootMeanSquaredError()]
                 )
    
    return model

model = get_compiled_model()
model.fit(X_train, y_train, epochs=15)

Epoch 1/15
1351/1351 [==============================] - 2s 1ms/step - loss: 537.0518 - root_mean_squared_error: 665.6535
Epoch 2/15
1351/1351 [==============================] - 2s 1ms/step - loss: 348.5305 - root_mean_squared_error: 503.9392
Epoch 3/15
1351/1351 [==============================] - 1s 1ms/step - loss: 344.8794 - root_mean_squared_error: 504.3200
Epoch 4/15
1351/1351 [==============================] - 2s 1ms/step - loss: 337.4964 - root_mean_squared_error: 496.9057
Epoch 5/15
1351/1351 [==============================] - 1s 1ms/step - loss: 331.0912 - root_mean_squared_error: 489.1979
Epoch 6/15
1351/1351 [==============================] - 2s 1ms/step - loss: 331.1851 - root_mean_squared_error: 487.9255
Epoch 7/15
1351/1351 [==============================] - 2s 1ms/step - loss: 327.4853 - root_mean_squared_error: 481.3303
Epoch 8/15
1351/1351 [==============================] - 2s 1ms/step - loss: 323.2343 - root_mean_squared_error: 471.7908A: 2s - loss: 328.9196 - root
Epo

In [18]:
# Evaluate model
model.evaluate(X_test, y_test)

338/338 [==============================] - 1s 1ms/step - loss: 302.0908 - root_mean_squared_error: 440.4026


[302.09075927734375, 440.40264892578125]

In [19]:
y_test 
print('Expected:')
print(y_test[:10])
print('What the model returns:')
print(model.predict(X_test[:10]))

Expected:
       Q_ds
45612  1310
20868  1371
6166   1157
24308   944
44175  1462
28566  1218
12172  1797
14290  1218
18959     0
9258   1371
What the model returns:
[[ 669.44995]
 [1300.6688 ]
 [1423.1588 ]
 [1209.1415 ]
 [1361.0962 ]
 [1269.7332 ]
 [1185.5516 ]
 [1216.9344 ]
 [ 279.11172]
 [1228.5212 ]]


### Our model has successfully been trained.

# Particle Filter Calculation

### Clean Data for Particle Filter

In [ ]:
def create_n_goal(df):
    # calculate n_goal
    # Shift up n_ups values
    # To do calculations for n_goal of ramp2 at time T, we need n_ups value of ramp1 at T-(1min)

    first_time = df['Time'].unique()[0] # first minute mark at which data was recorded
    first_ramp = df['Id'].unique()[0] # first ramp of the data
    last_time = df['Time'].unique()[-1]
    last_ramp = df['Id'].unique()[-1]

    new_df = df.copy()
    new_df['n_goal'] = 0
    new_df = new_df[(new_df['Time'] != first_time) & (new_df['Id'] != first_ramp)].reset_index(drop=True)

    temp_df = df.copy()
    temp_df = temp_df[(temp_df['Id'] != last_ramp) & (temp_df['Time'] != last_time)].reset_index(drop=True)
    temp_df = temp_df.rename(columns={'n_ups': 'n_goal'})

    new_df['n_goal'] = temp_df['n_goal']

    return new_df

# the larger the weight, the more efficient the q_ramp
def weight(n_ds, Q_ups, V_ups):
    n_ups = Q_ups/V_ups
    n_goal = 1.15 * n_ups
    temp = n_ds-n_goal
    R = 10 # arbitrary variable for normal Gaussian distribution 
    return (1/((2*math.pi*R)**(1/2))) * math.e**((-1)*((n_ds-n_goal)**2)/(2*R))

# mu: mean; best_rate
# sigma: standard deviation
def generate_range(mu, sigma=2):
    return np.random.normal(mu, sigma, 15).tolist()

def find_optimal_q_ramp(df, q_ramps):
    # load arbitraty q_ramp values

    V_ups = df['V_ups']
    Q_ups = df['Q_ups']
    q_exit = df['q_exit_prev']
    V_merge = df['V_merge']
    n_goal = df['n_goal']

    R = 1 # arbitrary
    cols = [] # list to store column names of weights

    for q_ramp in q_ramps:
        n_ds = (1/V_ups)*(Q_ups+(q_ramp/N))-(1/N)*(q_exit/V_merge)
        weight_val = weight(n_ds, Q_ups, V_ups)
        col = 'weight_'+str(q_ramp)
        cols.append(col)
        df.loc[:, col] = weight_val

    # choose the largest/best weight across multiple columns
    df = df.assign(q_ramp=df[cols].idxmax(axis=1))
    df = df[df['q_ramp'].notna()]
    df['q_ramp'] = df['q_ramp'].apply(lambda x: float(x.split('_')[1]))
    df = df[['Time', 'Id', 'postmile', 'V_ups', 'Q_ups', 'q_exit_prev', 'V_merge', 'n_ups', 'n_goal', 'q_ramp']]
    
    return df
    
def refine_q_ramp(df):
    
    q_ramps = df['q_ramp'].unique()
    new_df = pd.DataFrame()

    # for each q_ramp value
    for q_ramp in q_ramps:
        temp_df = df.loc[df['q_ramp'] == q_ramp]
        # generate a new set of q_ramps using normal distribution to fine-tune the q_ramp value
        normal_distribution = generate_range(q_ramp)
        temp_df = find_optimal_q_ramp(temp_df, normal_distribution)
        new_df = pd.concat([new_df, temp_df])

    new_df.sort_values(by=['Time', 'postmile'])
    
    return new_df

def particle_filter(raw):
    df = raw.copy()
    df = df[['Time', 'Id', 'postmile', 'V_ups', 'Q_ups', 'q_exit_prev', 'V_merge']]
    df.loc[:, 'n_ups'] = df['Q_ups'] / df['V_ups']
    
    # calculate n_goal
    df = create_n_goal(df)
    
    # calculate optimizing q_ramp from random q_ramp samples
    q_ramps = [1008,971,934,896,859,822,785,748,710,673,636,599,562,524,484]
    q_ramps = [i* 2 for i in q_ramps] # two ramps => double the discharge
    df = find_optimal_q_ramp(df, q_ramps)
    
    # calculate refined optimizing q_ramp from gaussian distribution
    df = refine_q_ramp(df)
    
    return df

In [ ]:
particle_filter(raw)

# Compare Particle Filter against TensorFlow

In [ ]:
calculation = particle_filter(raw).sample(1)
calculation[['V_ups', 'Q_ups', 'q_exit_prev', 'q_ramp']]

model_input = calculation[['Q_ups', 'V_ups', 'q_exit_prev', 'q_ramp']].reset_index(drop=True)
model_input

In [ ]:
def DL_guest_best(df):
    new_df = df.copy()
    
    q_ramps = np.arange(100, 2000, 0.5) * 2 # two ramps => double the discharge
    
    for i, q_ramp in enumerate(q_ramps):
        new_df.loc[i, :] = df.iloc[0, :]
        new_df.loc[i, 'q_ramp'] = q_ramp
                
    return q_ramps[np.argmin([sum(each) for each in model.predict(new_df).tolist()])]

In [ ]:
DL_guest_best(model_input)

# Visualization

In [ ]:
# libraries
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
toPlot = raw.copy()

time = "17:30:00"



# calculate distance between each ramp
toPlot = toPlot.sort_values('postmile')
toPlot['distance'] = toPlot[['postmile']].diff(periods=1)['postmile']
df[-1, 'distance'] = 0.0

# calculate travel time
toPlot['travel_time'] = toPlot['V_ups'] * toPlot['distance']

# calculate volume
toPlot['volume'] = toPlot['V_ups'] * toPlot['n_ds']

# calculate occupancy
toPlot['occupancy'] = (toPlot['n_ds']*(L+l))/(5280+(toPlot['n_ds']+l))

# drop unnecessary columns
toPlot = toPlot.drop(columns = ['Id', 'Q_ups', 'convtime', 'Time', 'q_exit', 'q_ramp', 'postmile'])

# rename columns
toPlot.rename(columns={"V_ups": "Speed(mph)", 
                       "distance": "Distance(mi)", 
                       "occupancy": "Occupancy(%)",
                       "volume": "Volume",
                       "travel_time": "Travel Time(min)"}, inplace=True)

original_toPlot = toPlot.copy()

In [ ]:
toPlot

In [ ]:
labels = [
          'Speed(mph)',
          'Occupancy(%) X 2',
          'Distance(mi) X 100',
          'Travel Time(min)',
          'Volume X 10'
          ]
toPlot['Occupancy(%)'] = toPlot['Occupancy(%)']*200
toPlot['Occupancy(%)'] = toPlot['Occupancy(%)']*200
toPlot['Distance(mi)'] = toPlot['Distance(mi)']*100
toPlot['Volume'] = toPlot['Volume']*1/10

toPlot.reset_index(inplace=True)


  
# Change the style of plot
plt.style.use('seaborn-dark')


# Set the background colors
fig = plt.figure()
fig.patch.set_facecolor('whitesmoke')

ax = fig.add_subplot(111)
ax.set_facecolor("whitesmoke")

# Initialize x_labels
x_label = toPlot.pop('Name')

# Initialize colors
colors = plotcolor = ['hotpink','orange','darkgoldenrod','lightseagreen','orchid']
 
# Add titles
plt.title("Highway CA-78", loc='left', fontsize=15, fontweight='bold', color='black')
plt.xlabel("Ramp, in the order of traffic flow", fontsize=10, fontweight='bold')
plt.xticks(rotation = 45)
# # Show the graph

num=0
for column, name, color in zip(toPlot.columns[1:], labels, colors): # start from 1 to avoid index from being plotted
    num+=1
    plt.plot(x_label, toPlot[column], marker='', color=color, linewidth=1.2, alpha=0.9, label=name)
    
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(9)

plt.axis([0, len(x_label), 0, 300])
plt.legend(loc=2, ncol=2, prop={'size': 9})

annotation = "data recorded on " + date + " " + time[:-3]
plt.figtext(0, -0.03, annotation, fontsize=7)


plt.tight_layout()
fig1 = plt.gcf()
plt.show()
plt.draw()
plt.gcf().subplots_adjust(bottom=0.45)
fig1.savefig('graph.png', dpi=500)

toPlot = original_toPlot

In [ ]:
# plot speed

# Change the style of plot
plt.style.use('seaborn-dark')

# Set the background colors
fig = plt.figure()
fig.patch.set_facecolor('whitesmoke')

ax = fig.add_subplot(111)
ax.set_facecolor("whitesmoke")

# Initialize colors
colors = plotcolor = ['hotpink','orange','darkgoldenrod','lightseagreen','orchid']

# Initialize x_labels
# defined in previous block

# Add titles
plt.title("Highway CA-78 Speed", loc='left', fontsize=15, fontweight='bold', color='black')
plt.xlabel("Ramp, in the order of traffic flow", fontsize=10, fontweight='bold')
plt.xticks(rotation = 45)

# plot graph
plt.plot(x_label, toPlot['Speed(mph)'], marker='', color='orange', linewidth=1.2, alpha=0.9, label='Speed(mph)')
    
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(9)

plt.axis([0, len(x_label), 0, 130])
plt.legend(loc=2, ncol=2, prop={'size': 9})

annotation = "data recorded on " + date + " " + time[:-3]
plt.figtext(0, -0.03, annotation, fontsize=7)


plt.tight_layout()
fig1 = plt.gcf()
plt.show()
plt.draw()
plt.gcf().subplots_adjust(bottom=0.45)
fig1.savefig('speed.png', dpi=500)

toPlot = original_toPlot

In [ ]:
# plot distance

# Change the style of plot
plt.style.use('seaborn-dark')

# Set the background colors
fig = plt.figure()
fig.patch.set_facecolor('whitesmoke')

ax = fig.add_subplot(111)
ax.set_facecolor("whitesmoke")

# Initialize colors
colors = plotcolor = ['hotpink','orange','darkgoldenrod','lightseagreen','orchid']

# Initialize x_labels
# x_label = toPlot.pop('Name')

# Add titles
plt.title("Highway CA-78 Distance", loc='left', fontsize=15, fontweight='bold', color='black')
plt.xlabel("Ramp, in the order of traffic flow", fontsize=10, fontweight='bold')
plt.xticks(rotation = 45)

# plot graph
plt.plot(x_label, toPlot['Distance(mi)'], marker='', color='orange', linewidth=1.2, alpha=0.9, label='Distance(mi)')
    
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(9)

plt.axis([0, len(x_label), 0, 3])
plt.legend(loc=2, ncol=2, prop={'size': 9})

annotation = "data recorded on " + date + " " + time[:-3]
plt.figtext(0, -0.03, annotation, fontsize=7)


plt.tight_layout()
fig1 = plt.gcf()
plt.show()
plt.draw()
plt.gcf().subplots_adjust(bottom=0.45)
fig1.savefig('distance.png', dpi=500)

toPlot = original_toPlot

In [ ]:
# plot distance

# Change the style of plot
plt.style.use('seaborn-dark')

# Set the background colors
fig = plt.figure()
fig.patch.set_facecolor('whitesmoke')

ax = fig.add_subplot(111)
ax.set_facecolor("whitesmoke")

# Initialize colors
colors = plotcolor = ['hotpink','orange','darkgoldenrod','lightseagreen','orchid']

# Initialize x_labels
# x_label = toPlot.pop('Name')

# Add titles
plt.title("Highway CA-78 Occupancy", loc='left', fontsize=15, fontweight='bold', color='black')
plt.xlabel("Ramp, in the order of traffic flow", fontsize=10, fontweight='bold')
plt.xticks(rotation = 45)

# plot graph
plt.plot(x_label, toPlot['Occupancy(%)'], marker='', color='orange', linewidth=1.2, alpha=0.9, label='Occupancy(%)')
    
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(9)

plt.axis([0, len(x_label), 0, 0.5])
plt.legend(loc=2, ncol=2, prop={'size': 9})

annotation = "data recorded on " + date + " " + time[:-3]
plt.figtext(0, -0.03, annotation, fontsize=7)


plt.tight_layout()
fig1 = plt.gcf()
plt.show()
plt.draw()
plt.gcf().subplots_adjust(bottom=0.45)
fig1.savefig('occupancy.png', dpi=500)

toPlot = original_toPlot

In [ ]:
# plot distance

# Change the style of plot
plt.style.use('seaborn-dark')

# Set the background colors
fig = plt.figure()
fig.patch.set_facecolor('whitesmoke')

ax = fig.add_subplot(111)
ax.set_facecolor("whitesmoke")

# Initialize colors
colors = plotcolor = ['hotpink','orange','darkgoldenrod','lightseagreen','orchid']

# Initialize x_labels
# x_label = toPlot.pop('Name')

# Add titles
plt.title("Highway CA-78 Volume", loc='left', fontsize=15, fontweight='bold', color='black')
plt.xlabel("Ramp, in the order of traffic flow", fontsize=10, fontweight='bold')
plt.xticks(rotation = 45)

# plot graph
plt.plot(x_label, toPlot['Volume'], marker='', color='orange', linewidth=1.2, alpha=0.9, label='Volume')
    
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(9)

plt.axis([0, len(x_label), 0, 3000])
plt.legend(loc=2, ncol=2, prop={'size': 9})

annotation = "data recorded on " + date + " " + time[:-3]
plt.figtext(0, -0.03, annotation, fontsize=7)


plt.tight_layout()
fig1 = plt.gcf()
plt.show()
plt.draw()
plt.gcf().subplots_adjust(bottom=0.45)
fig1.savefig('volume.png', dpi=500)

toPlot = original_toPlot

In [ ]:
# plot distance

# Change the style of plot
plt.style.use('seaborn-dark')

# Set the background colors
fig = plt.figure()
fig.patch.set_facecolor('whitesmoke')

ax = fig.add_subplot(111)
ax.set_facecolor("whitesmoke")

# Initialize colors
colors = plotcolor = ['hotpink','orange','darkgoldenrod','lightseagreen','orchid']

# Initialize x_labels
# x_label = toPlot.pop('Name')

# Add titles
plt.title("Highway CA-78 Travel Time", loc='left', fontsize=15, fontweight='bold', color='black')
plt.xlabel("Ramp, in the order of traffic flow", fontsize=10, fontweight='bold')
plt.xticks(rotation = 45)

# plot graph
plt.plot(x_label, toPlot['Travel Time(min)'], marker='', color='orange', linewidth=1.2, alpha=0.9, label='Travel Time(min)')
    
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(9)

plt.axis([0, len(x_label), 0, 200])
plt.legend(loc=2, ncol=2, prop={'size': 9})

annotation = "data recorded on " + date + " " + time[:-3]
plt.figtext(0, -0.03, annotation, fontsize=7)


plt.tight_layout()
fig1 = plt.gcf()
plt.show()
plt.draw()
plt.gcf().subplots_adjust(bottom=0.45)
fig1.savefig('travel_time.png', dpi=500)

toPlot = original_toPlot

In [ ]:
toPlot

In [ ]:
# # Dataframe
# toPlot = matrix.loc[matrix['Id'] == 201]
# toPlot = toPlot[['V_ups', 'Time']]
# toPlot['Time'] = pd.to_datetime(toPlot.Time)
# toPlot = toPlot.groupby(pd.Grouper(key='Time', freq='30min')).mean().dropna()
  
# # Change the style of plot
# plt.style.use('seaborn-darkgrid')
 
# # Create a color palette
# palette = plt.get_cmap('Set1')
 
# # Plot multiple lines
# num=0
# for column in toPlot.columns:
#     num+=1
#     plt.plot(toPlot.index.hour, toPlot[column], marker='', color=palette(num), linewidth=1, alpha=0.9, label=column)

# # Add legend
# plt.legend(loc=2, ncol=2)
 
# # Add titles
# plt.title("Speed VS Time", loc='left', fontsize=12, fontweight=0, color='orange')
# plt.xlabel("Hour")
# plt.xticks(rotation = 45)
# plt.ylabel("Speed (mph)")
# plt.ylim(0, 80)

# # Show the graph
# plt.show()

# UNEXPECTED VALUE RETURNED FROM MODEL:

In [ ]:
print('Expected:')
print(y_test)
print('What the model returns:')
print(model.predict(X_test))

In [ ]:
simple_test = df.iloc[1, :9].to_frame().transpose().reset_index()
simple_test

In [ ]:
test_df = pd.DataFrame()

q_ramps = [1008,971,934,896,859,822,785,748,710,673,636,599,562,524,484]   # hypothetical q_ramp
Q_ups = simple_test.loc[0, 'Q_ups']
V_ups = simple_test.loc[0, 'V_ups']
q_exit = simple_test.loc[0, 'q_exit']

test_df['q_ramp'] = q_ramps
test_df['Q_ups'] = Q_ups
test_df['V_ups'] = V_ups
test_df['q_exit'] = q_exit

test_df

### Our model accurately calculates Q_ds and V_ds. 
### We are now ready to compare our particle filter model with the DL model.

In [ ]:
model(test_df)